# Connecting To PostGres with Python

In this notebook, I want to showcase how the psycopg2 library can help us execute SQL-queries directly in a python terminal, IDE or notebook. 

The 'SuperMarketTransactions' csv-file has been uploaded to my localhost on PostGres. It includes data over fictional transactions in a pan-american supermarket, and we will use SQL to retreive insights on the behaviour of these customers


## Introduction - Defining the function

In [2]:
import psycopg2
import pandas as pd

Every PostGres database has the following specs that are needed to connect to it

In [3]:
DB_NAME = "TestData"
DB_USER = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

psycopg2 creates a "cursor" with which we can "fetch" data from the DB

In [4]:
try:
    conn = psycopg2.connect(database = DB_NAME, user = DB_USER,
                            host = DB_HOST, port = DB_PORT)
    print('connected')
except:
    print('not connected')

connected


In [6]:
conn.close()

Now that we've established a connection, lets write a function to retreive the data based an a Query (here called Q). Then, we fetch the raw SQL and turn it into a dataframe with it's accompanying column names  

In [5]:
def SQL_Query(Q):
    
    try:
        conn = psycopg2.connect(database = DB_NAME, user = DB_USER,
                            host = DB_HOST, port = DB_PORT)
        print('connected')
    except:
        print('not connected')
    
    cur = conn.cursor()
    cur.execute(Q)
    rows = cur.fetchall()
    Data = pd.DataFrame(rows)
    colname= [desc[0] for desc in cur.description]
    Data.columns = colname
    return(Data)
    curr.close()
    conn.close()

## Exploratory Analysis - Customers

Let's test it out! We start simply and call the average number of children of the customers by country they operate in.

In [7]:
QUERY = '''SELECT country,
                  round(avg(children),2) as avg_children
           FROM super 
           GROUP BY country 
           LIMIT 5 '''

In [8]:
SQL_Query(QUERY)

connected


,country,avg_children
0,Mexico,2.55
1,Canada,2.52
2,USA,2.52


It works! We see that even the alias was included. Let's try something more complicated, like incorporating a window function. In the Query below, we look at total revenue per city, ranked from highest to lowest, but also the ranking within country (Hidalgo bering the top performer in Mexico but only 13th overall.

In [9]:
QUERY1 = '''SELECT country, 
                   city, 
                   round(sum(revenue),2) as tot_rev,
                   RANK() OVER(ORDER BY round(sum(revenue),2)DESC),
                   RANK() OVER(PARTITION BY Country ORDER BY round(sum(revenue),2) DESC)
            FROM super
            GROUP BY city, country
            ORDER BY 3 DESC
            LIMIT 15'''

In [10]:
SQL_Query(QUERY1)

connected


,country,city,tot_rev,rank,rank
0,USA,Salem,11558.47,1,1
1,USA,Tacoma,9165.70,2,2
2,USA,Los Angeles,7281.37,3,3
3,USA,Seattle,7272.84,4,4
4,USA,Portland,6864.47,5,5
5,USA,Spokane,6729.45,6,6
6,USA,San Diego,6425.86,7,7
7,USA,Bremerton,6261.64,8,8
8,Mexico,Hidalgo,5992.43,9,1
9,USA,Beverly Hills,5449.34,10,9


What drives profitability in these stores? We can have a look at the customerbase to get a better understanding. Below we compute some metrics to compare.


In [11]:
Query2 =''' WITH sub AS (
                 SELECT *
                 FROM super
                 WHERE city IN ('Salem','Tacoma','Portland','Bremerton','Seattle','Los Angeles'))
            
            SELECT city,
                   round(avg(numerical_income),2) as avg_income,
                   round(avg(children),2) as avg_children,
                   round(avg(units_sold),2) as avg_basket_size,
                   round(avg(revenue),2) as avg_spend,
                   round((avg(CASE WHEN homeowner ='Y' THEN 1
                        ELSE 0 END) * 100),2) as home_ownership_percentage
            FROM sub
            GROUP BY city
            ORDER BY avg_spend DESC
            '''

In [12]:
SQL_Query(Query2)

connected


,city,avg_income,avg_children,avg_basket_size,avg_spend,home_ownership_percentage
0,Los Angeles,58089.89,2.49,4.12,13.64,61.61
1,Seattle,58026.07,2.43,4.16,13.54,58.29
2,Portland,56686.39,2.55,4.13,13.54,59.57
3,Tacoma,57097.24,2.61,4.19,13.30,59.51
4,Bremerton,57510.37,2.27,4.11,12.99,59.75
5,Salem,58104.79,2.50,4.07,12.89,55.85


Income over 55K/Year, >2.5 children and a high ration of home owners are traits for these high-revenue stores. 

## Analysis - KPI's 

But we want to dig deeper and see if we can define some KPI's for our stores performance. Lets query the information_schema to get an overview what we have to work with. We have 3 tables, 'margin', 'pop' and 'super'.

In [13]:
Query3 = '''SELECT table_name, 
                   column_name, 
                   data_type
            FROM information_schema.columns
            WHERE table_name IN ('margin','pop','super'); '''

In [14]:
SQL_Query(Query3)

connected


,table_name,column_name,data_type
0,pop,city,text
1,pop,population,numeric
2,margin,category,text
3,margin,margin,numeric
4,super,purchase_date,date
5,super,customer_id,integer
6,super,gender,text
7,super,marital_status,text
8,super,homeowner,text
9,super,children,integer


We need KPI's that accurately reflect the progress of our company's financials. We see that column 13, Product category, could be an interesting level to aggregate on and maybe define metrics. Lets
find the top 5 product categories and see how their profits evolved over time. The PostGres extension CROSSTAB will be used. 

In [15]:
Query4 = '''SELECT s.product_category, 
                   round(sum(s.revenue)*avg(m.margin),2) AS profit
            FROM super AS s
            INNER JOIN margin AS m
            ON s.product_category = m.category
            GROUP BY s.product_category
            ORDER BY 2 DESC
            LIMIT 5
            '''

In [16]:
SQL_Query(Query4)

connected


,product_category,profit
0,Vegetables,671.38
1,Snack Foods,655.70
2,Meat,335.64
3,Dairy,212.28
4,Bread,201.00


Alright! Lets compare how sales are doing between 2012 - 2014

In [17]:
Query4 = '''CREATE EXTENSION IF NOT EXISTS tablefunc;
            
            SELECT * FROM CROSSTAB($$
         
            SELECT product_category::text,
                   EXTRACT('Year' FROM purchase_date)::numeric as Year,
                   round(sum(s.revenue)*avg(m.margin),2)::numeric as profit
            FROM super as s
            INNER JOIN margin as m
            ON s.product_category = m.category
            WHERE s.product_category IN ('Vegetables','Snack Foods','Meat','Candy')
            GROUP BY s.product_category, Year
            ORDER BY s.product_category, Year $$ 

            ) AS ct (   
           
                      product_category text,
                      "2012" numeric,
                      "2013" numeric,
                      "2014" numeric
                                      ); '''
          

In [18]:
SQL_Query(Query4)

connected


,product_category,2012,2013,2014
0,Candy,1.32,72.64,114.32
1,Meat,1.83,121.67,212.14
2,Snack Foods,2.67,246.44,406.58
3,Vegetables,0.82,257.84,412.72


Our stores seem to be succesful! Most of our core product-categories have doubled over the past year. Is it because we've opened up new stores lately? 


In [19]:
Query5 = '''SELECT * FROM CROSSTAB($$
            SELECT 
                    country::text,
                    extract('Year' FROM purchase_date)::numeric as Year,
                    count(distinct city)::numeric
             FROM super
             GROUP BY country, Year
             ORDER BY country DESC, Year DESC $$)
             AS ct ( country text,
                     "2014" numeric,
                     "2013" numeric,
                     "2012" numeric)'''

In [20]:
SQL_Query(Query5)

connected


,country,2014,2013,2012
0,USA,13,13,10
1,Mexico,8,None,None
2,Canada,2,None,None


2014 was a big year with expansion into new markets! 10 new stores in 2 new countries where opened up. It's obvious that our profits are driven by new store-openings. Therefore, a KPI's based on the profitability of new stores could be defined. The US stores having been around for longer, we will have to define different KPI's dependant on the market. We will use:

- the average profitability/customer for Canada/Mexico
- the %YoY growth rate for stores with more than 1 year of history (USA)


In [21]:
Query6 =''' SELECT Country, City,
                   round(avg(s.revenue)*avg(m.margin)/count(s.customer_id) ,5)::numeric as profit
            FROM super as s
            INNER JOIN margin as m
            ON s.product_category = m.category
            WHERE country IN ('Mexico','Canada') 
            GROUP BY City, Country
            ORDER BY 3 DESC
            LIMIT 10
                  '''

In [22]:
SQL_Query(Query6)

connected


,country,city,profit
0,Mexico,Guadalajara,0.00614
1,Canada,Victoria,0.00571
2,Mexico,Mexico City,0.00445
3,Mexico,Acapulco,0.00265
4,Mexico,Orizaba,0.00259
5,Mexico,Camacho,0.00236
6,Canada,Vancouver,0.00181
7,Mexico,San Andres,0.00175
8,Mexico,Merida,0.00160
9,Mexico,Hidalgo,0.00137


In [23]:
Query7 ='''WITH sub AS(
                SELECT country,
                       city, 
                       EXTRACT('Year' FROM purchase_date) as Year, 
                       sum(revenue) as Rev
                FROM super
                GROUP BY city, country, Year)
            
            SELECT s1.country,
                   s1.city,
                   s1.Year,
                   s2.Year,
                   round((s2.Rev-s1.Rev)/s1.Rev,2)*100 as perc_change
            FROM sub as s1
            INNER JOIN sub as s2
            ON s1.city = s2.city 
            AND s1.Year = s2.Year-1
            WHERE s2.Year = 2014
            ORDER BY 5 DESC

                
                
                ''' 

In [24]:
SQL_Query(Query7)

connected


,country,city,year,year,perc_change
0,USA,Bellingham,2013.0,2014.0,33.00
1,USA,Spokane,2013.0,2014.0,11.00
2,USA,Walla Walla,2013.0,2014.0,-1.00
3,USA,Bremerton,2013.0,2014.0,-2.00
4,USA,Seattle,2013.0,2014.0,-4.00
5,USA,Beverly Hills,2013.0,2014.0,-5.00
6,USA,Tacoma,2013.0,2014.0,-8.00
7,USA,Yakima,2013.0,2014.0,-8.00
8,USA,Portland,2013.0,2014.0,-10.00
9,USA,San Diego,2013.0,2014.0,-17.00


Altough Bellingham and Spokane are outliers, the general trend in the us from last year is pretty grim. This quick overview tells us that the supermarkets financial health is heavily dependant on the growth it gets from new market oppportunities. 

Therefore, we should focus on which cities to open up new markets in! We can have a look at how our customer base in Mexico/Canada looks like as these are our priorities. 

In [25]:
Query8 = '''WITH sub AS (
            SELECT *
            FROM super
            WHERE country IN ('Canada','Mexico'))


            SELECT product_category,
                   round(avg(s.revenue)*avg(m.margin),2) AS avg_profit,
                   round(avg(numerical_income),2) AS avg_inc,
                   round(avg(children),2) AS avg_child,
                   round(AVG(CASE WHEN gender = 'M' THEN 1
                                  WHEN gender ='F' THEN 0
                                  ELSE NULL END *100))-50 as perc_more_M_than_F,
                   round(AVG(CASE WHEN marital_status = 'M' THEN 1
                                  WHEN marital_status ='S' THEN 0
                                  ELSE NULL END *100))-50 as perc_more_Mar_than_S

                   
            FROM pop as p
            INNER JOIN sub as s
            USING(city)
            INNER JOIN margin as m
            ON s.product_category = m.category
            GROUP BY product_category
            ORDER BY 2 DESC
            LIMIT 10'''

In [26]:
SQL_Query(Query8)

connected


,product_category,avg_profit,avg_inc,avg_child,perc_more_m_than_f,perc_more_mar_than_s
0,Side Dishes,0.96,80000.00,2.75,0,25
1,Seafood,0.91,60000.00,0.00,50,-50
2,Magazines,0.86,56666.67,3.33,50,-17
3,Candy,0.85,70769.23,2.77,-12,-4
4,Snack Foods,0.84,58064.52,2.87,-5,5
5,Meat,0.80,49523.81,2.10,12,-2
6,Pure Juice Beverages,0.80,53333.33,2.00,-50,-17
7,Packaged Vegetables,0.77,20000.00,4.00,50,50
8,Pain Relievers,0.76,90000.00,1.50,0,-50
9,Bread,0.72,57777.78,3.11,17,-39
